#1

##1.1

##1.2

Softmax puisqu'il y a plus de 2 classes (Sigmoid ou Softmax)
- Able to handle multiple classes 
- Useful for output neurons — typically Softmax is used only for the output layer, for neural networks that need to classify inputs into multiple categories.

Relu :
- ReLU activation function is widely used and is default choice as it yields better results.
- ReLU function should only be used in the hidden layers.



##1.3

In [ ]:
#importing the drive to google colab
from google.colab import drive
#reading data from drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
#Split the dataset
#Train : 70%
#Validation : 25%
#Train & Validation to Train the model
#Test : 5% to test the model at the end and deduce the Accuracy and the Confusion Matrix
#In each folder we find 3 classes Labeled 0, 1 and 2 ; {1 : "BEN", 2 : "CAN", 0 : "NOR"}
import os
import random
import shutil
class1 = {"min":1,"max":9215}
class2 = {"min":9216,"max":10103}
class3 = {"min":10104,"max":11218,}

def ta9ssim(basePath,path,fileList):
    if not os.path.isdir(path+"/0"):
        os.makedirs(path+"/0")
    if not os.path.isdir(path+"/1"):
        os.makedirs(path+"/1")
    if not os.path.isdir(path+"/2"):
        os.makedirs(path+"/2")

    for fichier in fileList:
        if fichier.endswith(".png"):
            x = int(fichier.split(".")[0])
            if(class1["min"] <= x <= class1["max"]):
                try:
                    shutil.move(basePath+"/" + fichier , path+"/0/" + fichier)
                except:
                    pass
            elif(class2["min"] <= x <= class2["max"]):
                try:
                    shutil.move(basePath+"/" + fichier , path+"/1/" + fichier)
                except:
                    pass
            elif(class3["min"] <= x <= class3["max"]):
                try:
                    shutil.move(basePath+"/" + fichier , path+"/2/" + fichier)
                except:
                    pass

pathLearn = "/content/gdrive/MyDrive/ddsmROI"
pathTest = "/content/gdrive/MyDrive/ddsmROI/ddtest"
pathValid = "/content/gdrive/MyDrive/ddsmROI/ddvalid"

filelist=os.listdir(pathLearn)
nbFiles = 11218
validFiles = random.choices(filelist, k=int(nbFiles*0.25))
filelist = [a for a in filelist if a not in validFiles]
testFiles = random.choices(filelist, k=int(nbFiles*0.05))
filelist = [a for a in filelist if a not in testFiles]
learnFiles = filelist

ta9ssim(pathLearn,pathTest,testFiles)
ta9ssim(pathLearn,pathValid,validFiles)
ta9ssim(pathLearn,pathLearn+"/ddslearn",learnFiles)

In [1]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


ds_train = image_dataset_from_directory(
    'ddsmROI/ddtest2', #/train 
    image_size=[128, 128]

)

ds_valid = image_dataset_from_directory(
    'ddsmROI/ddvalid', #/valid
    image_size=[128, 128]
   
)

ds_test = image_dataset_from_directory(
    'ddsmROI/ddtest', #/test
    image_size=[128, 128]
)

Found 123 files belonging to 3 classes.
Found 2486 files belonging to 3 classes.
Found 538 files belonging to 3 classes.


##kernel_size: 
- An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.

- A common choice is to keep the kernel size at 3x3 or 5x5. The first convolutional layer is often kept larger.

##padding: 
- one of "valid" or "same" (case-insensitive). "valid" means no padding. "same" results in padding with zeros evenly to the left/right or up/down of the input.


In [2]:
#Creating the Model

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([

    # First Convolutional Block 
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  # give the input dimensions in the first layer Here we are learning 
                  # a total of 32 filters and then we use Max Pooling to reduce the spatial dimensions of the output volume.
                  # [height, width, color channels(RGB)]
                  input_shape=[128,128,3]),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=6, activation="relu"),
    layers.Dense(units=3, activation="softmax"),
])
#model.summary()

In [3]:
#Total Number of Layers, including the input & the output
len(model.layers)

9

#Use sparse categorical crossentropy when your classes are mutually exclusive (e.g. when each sample belongs exactly to one class) and categorical crossentropy when one sample can have multiple classes or labels are soft probabilities (like [0.5, 0.3, 0.2]).

In [4]:
#Training the model with the training set and the validation set (10 Epochs)
import tensorflow as tf

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-06),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#verbose : shows the details of each epoch
history = model.fit(
    ds_valid,
    validation_data=ds_test,
    epochs=4,
    verbose=1,
)

Epoch 1/4
78/78 [==============================] - 39s 484ms/step - loss: 2.6795 - accuracy: 0.5286 - val_loss: 0.8504 - val_accuracy: 0.7955
Epoch 2/4
78/78 [==============================] - 41s 522ms/step - loss: 0.8345 - accuracy: 0.7792 - val_loss: 0.8228 - val_accuracy: 0.7881
Epoch 3/4
78/78 [==============================] - 37s 475ms/step - loss: 0.8166 - accuracy: 0.7767 - val_loss: 0.8116 - val_accuracy: 0.7937
Epoch 4/4
78/78 [==============================] - 39s 503ms/step - loss: 0.8058 - accuracy: 0.7808 - val_loss: 0.7980 - val_accuracy: 0.7993


In [5]:
#Testing the model with the testing Set
import numpy as np

pred = model.predict(ds_train)

#argmax returns l'indice of the max value of the specified axe
pred = np.argmax(pred, axis=1)

pred

4/4 [==============================] - 1s 90ms/step


array([0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2,
       2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [8]:
def predict_labels(model,test_ds):
    # Initialize Lists
    labels = []
    pred_labels = []
     # Make the test dataset an iterable object
    test_ds = iter(test_ds)
    # For each batch, append labels to both the true label array and predicted label array
    for images, temp_labels in test_ds:
        # Collect True Data Labels
        temp_labels = tf.constant(temp_labels).numpy()
        labels.extend(temp_labels)
         # Collect Prediction Labels
        prediction = model.predict(images,batch_size=32)
        temp_pred_labels = prediction.argmax(axis=-1)
        pred_labels.extend(temp_pred_labels)
    return labels, pred_labels


In [9]:
import sklearn.metrics as metrics
y_true_labels,ypred = predict_labels(model,ds_train)
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=ypred)
confusion_matrix

1/1 [==============================] - 0s 169ms/step


array([[48,  3,  0],
       [33,  4,  7],
       [24,  3,  1]], dtype=int64)

In [23]:
def calculate_accuracy(labels, pred_labels):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    total = len(labels)
     # Count for Confusion Matrix
    for i in range(total):
        if labels[i] == 1 and pred_labels[i] == 1:
            TP += 1 # True Positive
        elif labels[i] == 0 and pred_labels[i] == 0:
            TN += 1 # True Negative
        elif labels[i] == 0 and pred_labels[i] == 1:
            FP += 1 # False Positive
        elif labels[i] == 1 and pred_labels[i] == 0:
            FN += 1 # False Negative
        # Calculate Accuracy and return results
    accuracy = (TN + TP)/total
    return accuracy
y_true_labels,ypred = predict_labels(model,ds_train)
print("Accuracy : "+str(calculate_accuracy(y_true_labels,ypred)))

1/1 [==============================] - 0s 88ms/step
Accuracy : 0.42276422764227645


In [11]:
labels = {1 : "BEN", 2 : "CAN", 0 : "NOR"}
y_pred = [labels[k] for k in pred]
y_pred

['BEN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'CAN',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'CAN',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN',
 'NOR',
 'NOR',
 'CAN',
 'BEN',
 'NOR',
 'NOR',
 'NOR',
 'NOR',
 'BEN']

In [25]:
import gradio as gr
image=gr.inputs.Image(shape=(128,128))
label=gr.outputs.Label(num_top_classes=8)
labels = {1 : "BEN / benign", 2 : "CAN / malignant", 0 : "NOR / normal"}
def predict_img(img):
    print( model.predict(np.array([img,] )))
    return labels[np.argmax(model.predict(np.array([img,] )))]

gr.Interface(fn=predict_img,inputs=image,outputs=label,examples=["C:/Users/yosri/Desktop/New folder/ddsmROI/ddtest/0/38.png", "C:/Users/yosri/Desktop/New folder/ddsmROI/ddtest/0/91.png"], allow_flagging="never").launch()

C:\Users\yosri\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\yosri\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 25ms/step
[[9.7448051e-01 2.5335383e-02 1.8409510e-04]]
1/1 [==============================] - 0s 21ms/step
[[0.25298584 0.42014688 0.32686734]]
1/1 [==============================] - 0s 30ms/step
[[9.7448051e-01 2.5335383e-02 1.8409510e-04]]
1/1 [==============================] - 0s 36ms/step
